# VegET Testing

Imports

In [8]:
import VegET
from VegET import interpolate, daily_aggregate, utils, veg_et_model
#import cartoee as cee
import matplotlib.pyplot as plt
import ee
import ee.mapclient
from ipygee import *

Initialize EarthEngine

In [9]:
ee.Initialize()

Define date range

In [10]:
start_date = ee.Date('2003-04-01')
end_date = ee.Date('2003-11-01')

Define ROI 

In [11]:
# Filter to only include images within the colorado and utah boundaries (from ee-api/python examples)
polygon = ee.Geometry.Polygon([[
    [-109.05, 37.0], [-102.05, 37.0], [-102.05, 41.0],   # colorado
    [-109.05, 41.0], [-111.05, 41.0], [-111.05, 42.0],   # utah
    [-114.05, 42.0], [-114.05, 37.0], [-109.05, 37.0]]])

Define growing season months as integers. Note, filtering is inclusive.

In [12]:
g_season_begin = 4
g_season_end = 10

**NOTE**: for this case, the imagecollections are global or continent wide rasters. Ordinarily, the
imageCollections would need `.filterBounds()` to the ROI to subset to the images that intersect the
polygon. In this case, the filter does nothing since the images are continent/global scale.

Get ImageCollection used to calculate NDVI values. In this example, MODIS data are used.

In [13]:
ndvi_coll = ee.ImageCollection("MODIS/006/MOD09Q1").filterDate(start_date, end_date)\
    .filter(ee.Filter.calendarRange(g_season_begin, g_season_end, 'month'))\
    .map(lambda f: f.clip(polygon))
ndvi_coll = ndvi_coll.map(VegET.utils.getNDVI)

Get daily climate data (precip, eto, temp)

In [14]:
precip_eto_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(start_date, end_date)\
    .select('pr', 'eto', 'tmmn', 'tmmx').filter(ee.Filter.calendarRange(g_season_begin, g_season_end, 'month'))\
    .map(lambda f: f.clip(polygon))

# Add band for calculated mean daily temp
precip_eto_coll = precip_eto_coll.map(VegET.utils.dailyMeanTemp)
# Convert to Celsius
precip_eto_coll = precip_eto_coll.map(VegET.utils.kelvin2celsius).select(['pr', 'eto', 'tminC', 'tmaxC', 'tmeanC'])

VegET static inputs

In [15]:
# Specify canopy intercept image or imageCollection. NOTE: Assumes single band image
canopy_int = ee.Image('users/darin_EE/VegET/Interception').clip(polygon).double().rename('intercept')
# Get static Soil Water Holding Capacity grid (manually uploaded as GEE asset)
whc = ee.Image('users/darin_EE/VegET/WaterHoldingCapacity_mm').clip(polygon).double().rename('whc')
# Get static Soil Saturation image
soil_sat = ee.Image('users/darin_EE/VegET/SoilSaturation_mm').clip(polygon).double().rename('soil_sat')
# Get static Field Capacity image
fcap = ee.Image('users/darin_EE/VegET/FieldCapacity_mm').clip(polygon).double().rename('fcap')

# Create single static image with static inputs as bands
staticImage = canopy_int.addBands([whc, soil_sat, fcap])

Add static data to ndvi_coll as bands

In [16]:
ndvi_coll = ndvi_coll.map(VegET.utils.addStaticBands([staticImage]))

Daily interpolation. Primarily using methods developed in OpenET

In [18]:
# Create daily interpolated ndvi collection
ndvi_daily = interpolate.daily(precip_eto_coll, ndvi_coll)

# Add date band as 'time'
ndvi_daily = ee.ImageCollection(ndvi_daily.map(VegET.utils.add_date_band))

## Run VegET model

In [19]:
vegET_run = veg_et_model.vegET_model(ndvi_daily, polygon)

In [20]:
image = vegET_run.first()
print(image.bandNames().getInfo())
print(image.getInfo())

['swi', 'swe', 'snowpack']
{'type': 'Image', 'bands': [{'id': 'swi', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [289, 121], 'origin': [257, 178], 'crs': 'GEOGCS["unknown", \n  DATUM["unknown", \n    SPHEROID["Spheroid", 6378137.0, 298.257223563]], \n  PRIMEM["Greenwich", 0.0], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Longitude", EAST], \n  AXIS["Latitude", NORTH]]', 'crs_transform': [0.041666666666666664, 0.0, -124.78749996666667, 0.0, -0.041666666666666664, 49.42083333333334]}, {'id': 'swe', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}, {'id': 'snowpack', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}], 'properties': {'system:time_start': 1049176800000, 'system:index': '0'}}


View results

## Create a Map instance    

Arguments:   
- tabs: a tuple indicating which tabs to load in the map. Options are: Inspector, Layers, Assets, Tasks   
- kwargs: as this class inherits from ipyleaflet. Map it can accept all its arguments

In [21]:
Map = Map()

## Show map with method show
- Arguments
    - tabs: show tabs (bool)
    - layer_control: show a control for layers (bool)
    - draw_control: show a control for drawings (bool)
    - fullscrean: show fullscreen button (bool)

In [22]:
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Resize Map
Dimensions must be in pixel units

In [23]:
Map.setDimensions('90%', '500px')

## Define visualization parameters

In [24]:
visParam = {'bands': ['swi', 'swe', 'snowpack'], 'min': 0, 'max': 100}
visParamNDVI = {'bands': ['ndvi'], 'min': 0, 'max': 1}

## Add Layers   

In [25]:
Map.addLayer(image, visParam, name = 'VegET first')
#Map.addLayer(polygon, name = 'ROI')

## Add multiple images from a collection   

In [26]:
Map.addImageCollection(ndvi_daily.limit(5), visParamNDVI, namePattern = 'Daily NDVI for {system_date}')